In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

AutoML training tabular regression model for online prediction using BigQuery


This notebook simplifies the below notebook for training
*  https://colab.sandbox.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_tabular_regression_online_bq.ipynb?authuser=2#scrollTo=iFuEezd2CzIk

### Dataset

The dataset used for this tutorial is the GSOD dataset from [BigQuery public datasets](https://cloud.google.com/bigquery/public-data). The version of the dataset you use only the fields year, month and day to predict the value of mean daily temperature (mean_temp).

### Install Vertex AI SDK

In [ ]:
! pip3 install --upgrade --quiet google-cloud-aiplatform \
                                google-cloud-storage

## Configuration

### Authenticate your notebook environment

In [ ]:
import sys
from IPython.display import Markdown, display

PROJECT_ID="ai-hangsik"
LOCATION="us-central1"

# For only colab user, no need this process for Colab Enterprise in Vertex AI.
if "google.colab" in sys.modules:
    from google.colab import auth
    auth.authenticate_user(project_id=PROJECT_ID)

# set project.
!gcloud config set project {PROJECT_ID}

Updated property [core/project].


### Initialize Vertex AI SDK

In [ ]:
from google.cloud import aiplatform, bigquery
aiplatform.init(project=PROJECT_ID, location=LOCATION)

### Data preparation

#### Location of BigQuery training data.

In [ ]:
IMPORT_FILE = "bq://bigquery-public-data.samples.gsod"

### Create the Dataset

Next, create the dataset resource using the `create` method for the `TabularDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the dataset resource.
- `bq_source`: Alternatively, import data items from a BigQuery table into the dataset resource.

This operation may take several minutes.

In [ ]:
# https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.TabularDataset#google_cloud_aiplatform_TabularDataset_create

dataset = aiplatform.TabularDataset.create(
    display_name="NOAA historical weather data_unique online",
    bq_source=[IMPORT_FILE],
)

label_column = "mean_temp"

print(dataset.resource_name)

INFO:google.cloud.aiplatform.datasets.dataset:Creating TabularDataset
INFO:google.cloud.aiplatform.datasets.dataset:Create TabularDataset backing LRO: projects/721521243942/locations/us-central1/datasets/7080551967423987712/operations/4954796765132881920
INFO:google.cloud.aiplatform.datasets.dataset:TabularDataset created. Resource name: projects/721521243942/locations/us-central1/datasets/7080551967423987712
INFO:google.cloud.aiplatform.datasets.dataset:To use this TabularDataset in another session:
INFO:google.cloud.aiplatform.datasets.dataset:ds = aiplatform.TabularDataset('projects/721521243942/locations/us-central1/datasets/7080551967423987712')


projects/721521243942/locations/us-central1/datasets/7080551967423987712


In [ ]:
TRANSFORMATIONS = [
    {"auto": {"column_name": "year"}},
    {"auto": {"column_name": "month"}},
    {"auto": {"column_name": "day"}},
]

label_column = "mean_temp"

### Create and run training pipeline

In [ ]:
# # https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.AutoMLTabularTrainingJob

job = aiplatform.AutoMLTabularTrainingJob(
    display_name="job_automl_tabular_regression_online",
    optimization_prediction_type="regression",
    optimization_objective="minimize-rmse",
    column_transformations=TRANSFORMATIONS,
)

print(job)

#### Run the training pipeline

In [ ]:
# https://cloud.google.com/python/docs/reference/aiplatform/latest/google.cloud.aiplatform.AutoMLTabularTrainingJob#google_cloud_aiplatform_AutoMLTabularTrainingJob_run

model = job.run(
    dataset=dataset,
    model_display_name="automl_tabular_regression_online",
    training_fraction_split=0.6,
    validation_fraction_split=0.2,
    test_fraction_split=0.2,
    budget_milli_node_hours=8000,
    disable_early_stopping=False,
    target_column=label_column,
)

INFO:google.cloud.aiplatform.training_jobs:View Training:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/5130350083023306752?project=721521243942
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/721521243942/locations/us-central1/trainingPipelines/5130350083023306752 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/721521243942/locations/us-central1/trainingPipelines/5130350083023306752 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/721521243942/locations/us-central1/trainingPipelines/5130350083023306752 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud.aiplatform.training_jobs:AutoMLTabularTrainingJob projects/721521243942/locations/us-central1/trainingPipelines/5130350083023306752 current state:
PipelineState.PIPELINE_STATE_RUNNING
INFO:google.cloud

## Review model evaluation scores
After your model training has finished, you can review the evaluation scores for it using the `list_model_evaluations()` method.

In [ ]:
model_evaluations = model.list_model_evaluations()
if len(model_evaluations) > 0:
    eval_res = model_evaluations[0].to_dict()
    evaluation_metrics = eval_res["metrics"]
print(evaluation_metrics)

{'rootMeanSquaredError': 20.726614, 'meanAbsoluteError': 15.598564, 'meanAbsolutePercentageError': 9897303.0, 'rSquared': 0.2675524, 'rootMeanSquaredLogError': 0.84477913}


## Deploy the model

Next, deploy your model for online prediction. To deploy the model, you invoke the `deploy` method, with the following parameters:

- `machine_type`: The type of compute machine.

In [10]:
endpoint = model.deploy(machine_type="n1-standard-4")

INFO:google.cloud.aiplatform.models:Creating Endpoint
INFO:google.cloud.aiplatform.models:Create Endpoint backing LRO: projects/721521243942/locations/us-central1/endpoints/2128312563258097664/operations/1045544745226469376
INFO:google.cloud.aiplatform.models:Endpoint created. Resource name: projects/721521243942/locations/us-central1/endpoints/2128312563258097664
INFO:google.cloud.aiplatform.models:To use this Endpoint in another session:
INFO:google.cloud.aiplatform.models:endpoint = aiplatform.Endpoint('projects/721521243942/locations/us-central1/endpoints/2128312563258097664')
INFO:google.cloud.aiplatform.models:Deploying model to Endpoint : projects/721521243942/locations/us-central1/endpoints/2128312563258097664
INFO:google.cloud.aiplatform.models:Deploy Endpoint model backing LRO: projects/721521243942/locations/us-central1/endpoints/2128312563258097664/operations/9156527674120732672
INFO:google.cloud.aiplatform.models:Endpoint model deployed. Resource name: projects/72152124394

## Send a online prediction request

Send a online prediction to your deployed model.

### Make test item

You use synthetic data as a test data item. Don't be concerned that you're using synthetic data -- you just want to demonstrate how to make a prediction.

In [11]:
INSTANCE = {"year": "1932", "month": "11", "day": "6"}

### Make the prediction

Now that your `Model` resource is deployed to an `Endpoint` resource, you can do online predictions by sending prediction requests to the `Endpoint` resource.

#### Request

The format of each instance is:

    [feature_list]

Since the predict() method can take multiple items (instances), send your single test item as a list of one test item.

#### Response

The response from the predict() call is a Python dictionary with the following entries:

- `ids`: The internal assigned unique identifiers for each prediction request.
- `value`: The predicted value for each prediction.
- `deployed_model_id`: The Vertex AI identifier for the deployed `Model` resource which did the predictions.

In [12]:
instances_list = [INSTANCE]

prediction = endpoint.predict(instances_list)
print(prediction)

Prediction(predictions=[{'value': 45.2983283996582, 'lower_bound': 12.78389739990234, 'upper_bound': 61.65780258178711}], deployed_model_id='2326501733187977216', metadata=None, model_version_id='1', model_resource_name='projects/721521243942/locations/us-central1/models/7398170640118710272', explanations=None)


## Undeploy the model

When you are done doing predictions, you undeploy the model from the `Endpoint` resouce. This deprovisions all compute resources and ends billing for the deployed model.

In [ ]:
endpoint.undeploy_all()

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Dataset
- Model
- Endpoint
- AutoML Training Job

In [ ]:
# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the endpoint using the Vertex endpoint object
endpoint.delete()

# Delete the AutoML trainig job
job.delete()

delete_bucket = False  # set True to delete bucket

if delete_bucket:
    ! gsutil rm -r $BUCKET_URI